<a href="https://colab.research.google.com/github/psaraswatshell/AiAgile/blob/master/Bert_Product_Zema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf apex
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6593 (delta 0), reused 2 (delta 0), pack-reused 6589
Receiving objects: 100% (6593/6593), 13.71 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (4382/4382), done.


In [0]:
!cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-y2o3fy8d
Created temporary directory: /tmp/pip-req-tracker-ycppyntm
Created requirements tracker '/tmp/pip-req-tracker-ycppyntm'
Created temporary directory: /tmp/pip-install-1d39q5g1
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-t06p5f9w
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-ycppyntm'
    Running setup.py (path:/tmp/pip-req-build-t06p5f9w/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.4.0
    running egg_info
    creating /tmp/pip-req-build-t06p5f9w/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-t06p5f9w/pip-egg-info/apex.egg-info/PKG-INFO
  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -r './drive/My Drive/Colab Notebooks/requirements.txt'

     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 573kB 16.9MB/s 
     |████████████████████████████████| 3.8MB 22.9MB/s 
     |████████████████████████████████| 1.0MB 46.6MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
     |████████████████████████████████| 112kB 58.0MB/s 
     |████████████████████████████████| 552kB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=de798ebc544b06f536997ff963c8884da62cd4a93a149ba67a651931bd958068
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which is incompatible.


In [0]:
!pip install fast-bert

     |████████████████████████████████| 81kB 5.6MB/s 


In [0]:
from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split

import datetime

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [0]:
torch.cuda.empty_cache()

In [0]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [0]:
DATA_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/')
LABEL_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/')

AUG_DATA_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/data-aug/')

MODEL_PATH=Path('./drive/My Drive/Colab Notebooks/ap_product/models')
LOG_PATH=Path('./drive/My Drive/Colab Notebooks/ap_product/logs/')
MODEL_PATH.mkdir(exist_ok=True)

model_state_dict = None

# BERT_PRETRAINED_PATH = Path('../../bert_models/pretrained-weights/cased_L-12_H-768_A-12/')
BERT_PRETRAINED_PATH = Path('/content/bert_models/pretrained-weights/uncased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/uncased_L-24_H-1024_A-16/')
# FINETUNED_PATH = Path('../models/finetuned_model.bin')
FINETUNED_PATH = None
# model_state_dict = torch.load(FINETUNED_PATH)

LOG_PATH.mkdir(exist_ok=True)

OUTPUT_PATH = MODEL_PATH/'output'
OUTPUT_PATH.mkdir(exist_ok=True)

In [0]:
args = Box({
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "product_classsification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": OUTPUT_PATH,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 16,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "warmup_proportion": 0.0,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "fp16_opt_level": "O1",
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 500,
    "logging_steps": 50,
    "eval_all_checkpoints": True,
    "overwrite_output_dir": True,
    "overwrite_cache": False,
    "seed": 42,
    "loss_scale": 128,
    "task_name": 'intent',
    "model_name": 'xlnet-base-cased',
    "model_type": 'xlnet'
})

In [0]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [0]:
logger.info(args)

04/27/2020 02:15:35 - INFO - root -   {'run_text': 'multilabel toxic comments with freezable layers', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('drive/My Drive/Colab Notebooks/ap_product/logs'), 'full_data_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product'), 'data_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product'), 'task_name': 'intent', 'no_cuda': False, 'bert_model': PosixPath('/content/bert_models/pretrained-weights/uncased_L-12_H-768_A-12'), 'output_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product/models/output'), 'max_seq_length': 512, 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 16, 'learning_rate': 5e-05, 'num_train_epochs': 6, 'warmup_proportion': 0.0, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'fp16_opt_level': 'O1', 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'max_steps': -1, 'warmup_steps': 500, '

In [0]:
# tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=args['do_lower_case'])

In [0]:
device = torch.device('cuda')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [0]:
label_cols = ["Natural Gas", "Crude Oil", "Diesel", "Gasoline", "Coal", "Propane", "Fuel Oil", "PropaneButane", "Base Oil", "Benzene", "Styrene", "Butadiene", "Propylene", "Gasoil", "Isobutylene", "Polybutadiene", "Acrylonitrite", "Ethylene", "Phenol", "Cyclohexane", "Ehtylene", "Mogas", "Jet Fuel", "Butane", "Isobutane", "Rig Count", "Naphtha", "VGO", "Cumene", "Ethane", "EthanePropane", "Marine Fuel"]

In [0]:
databunch = BertDataBunch(args['data_dir'], LABEL_PATH, args.model_name, train_file='train.csv', val_file='val.csv',
                          test_data='test.csv',
                          text_col="Text", label_col=label_cols,
                          batch_size_per_gpu=args['train_batch_size'], max_seq_length=args['max_seq_length'], 
                          multi_gpu=args.multi_gpu, multi_label=True, model_type=args.model_type)

04/27/2020 02:15:42 - INFO - filelock -   Lock 140206188624752 acquired on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad.lock
04/27/2020 02:15:42 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpg8dic4kf



04/27/2020 02:15:43 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json in cache at /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 02:15:43 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 02:15:43 - INFO - filelock -   Lock 140206188624752 released on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad.lock
04/27/2020 02:15:43 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /ro


04/27/2020 02:15:44 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model in cache at /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8
04/27/2020 02:15:44 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8
04/27/2020 02:15:44 - INFO - filelock -   Lock 140204714768424 released on /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock
04/27/2020 02:15:44 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model from cache at /root/.cache/tor

In [0]:
databunch.train_dl.dataset[0][3]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
# train_df.head(20)
# databunch = BertDataBunch.load(args['data_dir'])

In [0]:
num_labels = len(databunch.labels)
num_labels

32

In [0]:
# databunch.train_dl.dataset[10]

In [0]:
# torch.distributed.init_process_group(backend="nccl", 
#                                      init_method = "tcp://localhost:23459", 
#                                      rank=0, world_size=1)

In [0]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

In [0]:
learner = BertLearner.from_pretrained_model(databunch, args.model_name, metrics=metrics, 
                                            device=device, logger=logger, output_dir=args.output_dir, 
                                            finetuned_wgts_path=FINETUNED_PATH, warmup_steps=args.warmup_steps,
                                            multi_gpu=args.multi_gpu, is_fp16=args.fp16, 
                                            multi_label=True, logging_steps=0)



04/27/2020 02:16:01 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 02:16:01 - INFO - transformers.configuration_utils -   Model config XLNetConfig {
  "_num_labels": 32,
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bad_words_ids": null,
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"


04/27/2020 02:16:16 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin in cache at /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac
04/27/2020 02:16:16 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac
04/27/2020 02:16:16 - INFO - filelock -   Lock 140204722405048 released on /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock
04/27/2020 02:16:16 - INFO - transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin from cache at /r

In [0]:
learner.fit(args.num_train_epochs, args.learning_rate, validate=True)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
04/27/2020 02:17:20 - INFO - root -   ***** Running training *****
04/27/2020 02:17:20 - INFO - root -     Num examples = 8703
04/27/2020 02:17:20 - INFO - root -     Num Epochs = 6
04/27/2020 02:17:20 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
04/27/2020 02:17:20 - 

RuntimeError: ignored

In [0]:
learner.validate()

In [0]:
learner.save_model()

In [0]:
learner.predict_batch(list(pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/predict.csv')['Text'].values))

In [0]:
from fast_bert.prediction import BertClassificationPredictor


In [0]:
predictorProduct = BertClassificationPredictor(
				model_path='./drive/My Drive/Colab Notebooks/ap_product/models/output/model_out',
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=True,
				model_type='xlnet',
				do_lower_case=True)


In [0]:
predictorProduct.predict_batch(list(pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/predict2.csv')['Text'].values))

In [0]:
df = pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/Argus_cleaned_all.csv')
df.head()

In [0]:
for i, items in df.iterrows():
  predictedProduct = predictorProduct.predict(items['NLPText'])
  print(predictedProduct)
  df.at[i, 'Product']= predictedProduct[0][0]

In [0]:
predictorGeo = BertClassificationPredictor(
				model_path='./drive/My Drive/models/output/model_out',
				label_path='./drive/My Drive/Colab Notebooks/bert-data', # location for labels.csv file
				multi_label=True,
				model_type='xlnet',
				do_lower_case=True)

In [0]:
for i, items in df.iterrows():
  predictedGeo = predictorGeo.predict(items['NLPText'])
  print(predictedGeo)
  df.at[i, 'Geography']= predictedGeo[0][0]

In [0]:
df.to_csv('./drive/My Drive/Colab Notebooks/ap_product/Argus_NLP_Deep_Model_Bert.csv')